# Individual Assignment

In [1]:
import pandas as pd

crimestats = pd.read_csv("https://raw.githubusercontent.com/ethan2411/Data-603-604/main/604%20Data/Community_Crime_Statistics.csv")
crimestats.head(5)

,Sector,Community Name,Category,Crime Count,Date,Year,Month,ID,Resident Count,Community Center Point,Calgary Communities,Ward Boundaries 2013-2017,Ward Boundaries,City Quadrants
0,NaN,14V,Break & Enter - Commercial,1,2019/04,2019,APR,2019-APR-14V-Break & Enter - Commercial,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,02A,Assault (Non-domestic),1,2019/12,2019,DEC,2019-DEC-02A-Assault (Non-domestic),NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,10A,Theft FROM Vehicle,1,2019/01,2019,JAN,2019-JAN-10A-Theft FROM Vehicle,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,10C,Theft OF Vehicle,1,2019/11,2019,NOV,2019-NOV-10C-Theft OF Vehicle,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,14V,Theft FROM Vehicle,1,2019/01,2019,JAN,2019-JAN-14V-Theft FROM Vehicle,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#print the sum of NaN values in each column, maybe I can just drop some of the columns
print(len(crimestats))
print(crimestats.isna().sum())
#Don't include Resident Count in database, theyre all null

11857
Sector                          70
Community Name                   0
Category                         0
Crime Count                      0
Date                             0
Year                             0
Month                            0
ID                               0
Resident Count               11857
Community Center Point          70
Calgary Communities             70
Ward Boundaries 2013-2017       70
Ward Boundaries                 70
City Quadrants                  70
dtype: int64


### Connecting to database

In [2]:
#in cmd do mysql.connector or mysql-connector-python
import mysql.connector
from mysql.connector import errorcode

filepath = "C:/Users/ethan/Downloads/Data 604/Term Project/password.txt"

with open(filepath) as f:
    passw = f.read()
    
# attempt a connection
myconnection = mysql.connector.connect(user='ethan_scott', 
                                       password=passw,
                                       host='datasciencedb2.ucalgary.ca', 
                                       database='ethan_scott',
                                       allow_local_infile=True)
myconnection

### Creating table in SQL

In [31]:
# CREATE TABLE STATEMENT
create_statement = '''create table ethan_scott.crimestats (
    Sector varchar(4),
    CommunityName varchar(3) NOT NULL,
    Category varchar(40),
    CrimeCount int,
    Date varchar (7),
    Year varchar(4),
    Month varchar(3),
    ID varchar(100),
    CommunityCenterPoint point,
    CalgaryCommunities int,
    WardBoundaries20132017 int,
    WardBoundaries int,
    CityQuadrants int
    );'''

# now we'll create a cursor and run our create statement
create_cursor = myconnection.cursor()
try:
    create_cursor.execute(create_statement)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
        print("Ooops! We already have that table")
    else:
        print(err.msg)
else:
    print("table created successfully!")

create_cursor.close()

table created successfully!


True

# Using Python to insert data

Because we have already built a dataframe with data from our CSV, we are going to use this datafrarame by reading each line, and writing each line into its own insert command.

How would you re-write the block of code below to make it more efficient?

In [32]:
insertCursor = myconnection.cursor()

columnString = "`,`".join([str(currentColumn) for currentColumn in crimestats.columns.tolist()])
#print (columnString)

# inserting rows one by one from the DataFrame is sufficient for now
for i, currentRow in crimestats.iterrows():
    #print (tuple(currentRow))
    insertCommand = "INSERT INTO `crimestats` (`" + columnString + "`) VALUES (" + "%s,"*(len(currentRow)-1) + "%s)"
    insertCursor.execute(insertCommand, tuple(currentRow))
    
myconnection.commit()

insertCursor.close()

ProgrammingError: 1054 (42S22): Unknown column 'nan' in 'field list'

## Deleting tables

In [30]:
#Use the DELETE statement without specifying a WHERE clause
# If the statement executes, the table continues to exist (still can insert rows into it), but it's empty. 
# All existing views and authorizations on the table remain intact when using DELETE.

# uncomment if you would like to remove table
deletecursor = myconnection.cursor()
# Warning - if you DROP the table it will be removed completely and it will need to be created from scratch
sql = "DROP TABLE IF EXISTS crimestats;"
#sql = "DELETE FROM crimestats;"
deletecursor.execute(sql)
deletecursor.close()

True

# Using Python to retrieve data

You can also use cursors to read data from a database table. It is helpful to specify what kind of result set you would like the cursor to return. Try each of the following for the second arguement:
* `raw=True`
* `dictionary=True`
* `named_tuple=True`

In [31]:
# try changing the second argument in this method call.
read_cursor = myconnection.cursor(buffered=True, dictionary=True)

query_string = ("SELECT library FROM library_locations;")

read_cursor.execute(query_string)

for (library_value) in read_cursor:
    print(library_value)
    
read_cursor.close()

{'library': 'W.R. Castell Central Library'}
{'library': 'Alexander Calhoun Library'}
{'library': 'Bowness Library'}
{'library': 'Fish Creek Library'}
{'library': 'Forest Lawn Library'}
{'library': 'Glenmore Square Library'}
{'library': 'Louise Riley Library'}
{'library': 'Memorial Park Library'}
{'library': 'Nose Hill Library'}
{'library': 'Shawnessy Library'}
{'library': 'Signal Hill Library'}
{'library': 'Southwood Library'}
{'library': 'Judith Umbach Library'}
{'library': 'Village Square Library'}
{'library': 'Crowfoot Library'}
{'library': 'Country Hills Library'}
{'library': 'Saddletowne Library'}
{'library': 'Westbrook Library'}
{'library': 'W.R. Castell Central Library'}
{'library': 'Alexander Calhoun Library'}
{'library': 'Bowness Library'}
{'library': 'Fish Creek Library'}
{'library': 'Forest Lawn Library'}
{'library': 'Glenmore Square Library'}
{'library': 'Louise Riley Library'}
{'library': 'Memorial Park Library'}
{'library': 'Nose Hill Library'}
{'library': 'Shawnessy Libr

True

# Parameters in SQL

In [32]:
import datetime

size_cursor = myconnection.cursor(buffered=True, dictionary=True)
library_sizes = [0, 10000, 25000, 100000]

query_string = ("SELECT library FROM library_locations where square_feet < %s ;")
for current_size in library_sizes:
    print ("Number of libraries with square feet less than", current_size)
    size_cursor.execute(query_string, (current_size,))
    print (size_cursor.rowcount)

size_cursor.close()

Number of libraries with square feet less than 0
0
Number of libraries with square feet less than 10000
30
Number of libraries with square feet less than 25000
75
Number of libraries with square feet less than 100000
85


True

In [33]:
# CLEANUP: always remember to release the resources you have used on the server. Always run this cell last!
myconnection.close()